In [2]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from attackers.decoding_attacker import dec_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
#from solvers.simulated_annealing import simulated_annealing
#from solvers.aps_gibbs import aps_gibbs
from solvers.nn_RS.nn_RS import *
#from solvers.aps_gibbs_class import *

from solvers.aps_gibbs_class_exp_decay import *
from solvers.random_mc_ennumeration import *

from scipy.special import softmax

In [3]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])
emission   = np.array([[0.699, 0.05, 0.1, 0.05, 0.1, 0.001],
                    [0.001, 0.1, 0.1, 0.299, 0.3, 0.2],
                       [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]])

rho_probs = 0.95*np.ones(6)

x_vector = np.array([ [4], [3], [5], [3], [4] ])

In [2]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])

emission   = np.array([[0.699, 0.05, 0.1, 0.05, 0.1, 0.001],
                    [0.001, 0.1, 0.1, 0.299, 0.3, 0.2],
                       [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]])

rho_probs = 0.95*np.ones(6)

x_vector = np.array([ [5], [3], [4], [3], [5] ])

In [4]:
hmm = HMM(n_components=3, n_obs=6 )
hmm.emissionprob_ = emission
hmm.transmat_     = transition
hmm.startprob_    = priors

In [5]:
V, seq = hmm.nu(x_vector)
seq

array([1, 1, 1, 1, 1])

In [5]:
attacker = ss_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 1.0 ,w2 = 0.0 , t = 3, state= 0, c=1, k_value=10000)

In [55]:
attacker = dec_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 100.0 ,w2 = 0.0 , seq = np.array([2,0,0,0,2]), k_value=10000)


# Test

In [56]:
 z_opt = np.array([[0,1,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [0,0,0,0,0,1]])
attack_obs= attacker.attack_X(np.ones_like(z_opt), z_opt)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([0, 0, 0, 0, 2])

In [57]:
attacker.expected_utility(z_opt, N=10000)

893.827658565778

In [58]:
z_opt = np.array([[0,0,0,0,0,1], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [0,0,0,0,0,1]])
attack_obs = attacker.attack_X(np.ones_like(z_opt), z_opt)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([2, 0, 0, 0, 2])

In [59]:
attacker.expected_utility(z_opt, N=10000)

881.5315952341298

# Full ennumeration

In [28]:
z_star, z_vec = MC_enumeration(attacker, 10)

Percentage completed: 0.0
Percentage completed: 12.86
Percentage completed: 25.72
Percentage completed: 38.58
Percentage completed: 51.44
Percentage completed: 64.3
Percentage completed: 77.16
Percentage completed: 90.02


In [30]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([2, 0, 2, 0, 2])

In [ ]:
np.argsort(-z_vec)[:10]

In [9]:
attacker.expected_utility(Z_set[0], 10000)

2.989512134961151

In [21]:
Z_set[0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

# Simulated annealing

In [ ]:
z_star = simulated_annealing(attacker, 200, verbose=True)

In [9]:
attacker.expected_utility(z_star, N=100)

594.2007039682692

In [10]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))

In [11]:
seq

array([0, 0, 0, 0, 0])

# Gibbs - APS - Exp. Decay

In [41]:
from solvers.aps_gibbs_class_exp_decay import *
aps = aps_gibbs(attacker)

In [ ]:
z_star, quality = aps.iterate(simulation_seconds=20)

In [42]:
quality

583.1700903629143

In [43]:
z_star

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [ ]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

# Gibbs - APS

In [8]:
from solvers.aps_gibbs_class import aps_gibbs
aps = aps_gibbs(attacker, np.arange(500, 10000, 10))

In [9]:
z_star, quality = aps.iterate(simulation_seconds=10)

In [10]:
quality

588.0979225405633

In [11]:
z_star

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [37]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([0, 0, 0, 0, 0])

# RS with Neural Network

In [10]:
from solvers.nn_RS.nn_RS import *
rs = nn_RS(attacker, flag='SA', RS_iters=5000, mcts_iters=50, sa_iters=100, eps=0.01, lr=0.05)

In [11]:
z_star, quality = rs.iterate(simulation_seconds=20)

In [12]:
quality

582.8687781846639

In [15]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([0, 0, 0, 0, 0])

In [15]:
rs = nn_RS(attacker, flag='MCTS', RS_iters=5000, mcts_iters=50, sa_iters=100, eps=0.01, lr=0.5)
z_star, quality = rs.iterate(simulation_seconds=20)

In [16]:
quality

589.1479986772996

# Random MC Ennumeration

In [12]:
rmc = random_mc_ennumeration(attacker)

In [16]:
z_star, quality = rmc.iterate(simulation_seconds=20)

In [17]:
quality

586.1784824075855

In [18]:
z_star

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.]])

# Full APS - Old

In [197]:
def propose_z(z):
    z_new = z.copy()
    idx = np.random.choice(np.arange(z.shape[0]))
    candidates  = np.diag( np.ones(z.shape[1]) )
    candidates  = candidates[~np.all(candidates == z[idx], axis=1)]

    z_new[idx] = candidates[ np.random.choice(np.arange(candidates.shape[0])) ]
    return z_new

def update_z(H, hmm, z_new, z_old, old_value, k_value):
    
    transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = H, k = k_value)
    emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = H, k = k_value)
    priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = H, k = k_value)
    
    hmms = []
    rho_matrix_samples = []
    
    value = 0
    
    for h in range(H):
        
        hmm_n = HMM(hmm.n_components, hmm.n_obs)
        hmm_n.startprob_ = priors_vec_list[h].reshape(-1)
        hmm_n.transmat_ = transtion_mat_list[h]
        hmm_n.emissionprob_ = emission_mat_list[h]
        rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
        
        hmms.append(hmm_n)
        rho_matrix_samples.append(rho_matrix)
        
        y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_new, 
                             rho_matrix = rho_matrix).astype('int')
        
        value += np.log( 
            attacker.utility(hmm = hmm_n, z_matrix = z_new, 
                             x_obs_vector = x_vector, y_t = y_t) + 6 )
        
    value /= H

    prob =  np.exp(H*value - H*old_value)

    
    if np.random.uniform() < prob:
        
        print("Accept!")
        print(prob)
        print(z_new)
        print(z_old)
        
        return z_new, value
    
    else:
        
        return z_old, old_value

In [211]:
z_init = Z_set[ np.random.choice(Z_set.shape[0]) ]

z_init =np.array([[0., 1., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.]])


hh = 15000
k_value = 1000000
##
transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = hh, k = k_value)
emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = hh, k = k_value)
priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = hh, k = k_value)
rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

hmms = []
rho_matrix_samples = []

value = 0
for temp in range(hh):

    hmm_n = HMM(hmm.n_components, hmm.n_obs)
    hmm_n.startprob_ = priors_vec_list[temp].reshape(-1)
    hmm_n.transmat_ = transtion_mat_list[temp]
    hmm_n.emissionprob_ = emission_mat_list[temp]
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

    hmms.append(hmm_n)
    rho_matrix_samples.append(rho_matrix)

    y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_init, rho_matrix = rho_matrix).astype('int')
    value += np.log( 
        attacker.utility(hmm = hmm_n, z_matrix = z_init, x_obs_vector = x_vector, y_t = y_t) + 6 )
    

value /= hh
value   


1.9443352596947014

In [212]:
for temp in np.arange(15000, 20000, 250):

    z_new = propose_z(z_init)
    z_init, value = update_z(temp, hmm, z_new, z_init, value, k_value)
    
    print(temp)
    print(z_init)

   


15000
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15250
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15500
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Accept!
1.524466408672834
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15750
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16000
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16250
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16500
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]


KeyboardInterrupt: 